# DBMaint: Delete Everything


In [1]:
import os
import subprocess
import boto3
import json
import datetime

import pykx as kx

from env import *
from config import *
from managed_kx import *

# ----------------------------------------------------------------

# Selective Deletion or All
#DELETE_ALL=True
DELETE_ALL=False

DELETE_CLUSTERS=True
DELETE_SCALING_GROUP=False
DELETE_DATAVIEW=False
DELETE_VOLUME=False
DELETE_DATABASE=True

if DELETE_DATABASE:
    DELETE_DATAVIEW=True

if DELETE_VOLUME:
    DELETE_DATAVIEW=True

if DELETE_ALL:
    DELETE_CLUSTERS=True
    DELETE_SCALING_GROUP=True
    DELETE_DATAVIEW=True
    DELETE_VOLUME=True
    DELETE_DATABASE=True

In [2]:
# create finspace client
session = boto3.Session()
client = get_client(session=session)

## Delete all resources

In [3]:
# Clusters
#-------------------------------------------------------------------------------------------
if DELETE_CLUSTERS:
    print("Deleting Clusters...")

    delete_status = ['RUNNING', 'CREATING', 'ACTIVE']

    # Clusters
    for c in all_clusters:
        this_c = get_kx_cluster(client=client, clusterName=c)
        if this_c is None:
            print(f"Not Found: {c}")
            continue
        elif this_c['status'] not in delete_status:
            print(f"Cannot Delete: {c} Status: {this_c['status']}")
            continue

        print(f"\tDelete: {c}")
        client.delete_kx_cluster(environmentId=ENV_ID, clusterName=c)
        print(f"\tDeleted: {c}")

    print(f"** Deleted {len(all_clusters)} Clusters **")

# Dataview
#-------------------------------------------------------------------------------------------
if DELETE_DATAVIEW:
    print()
    print("Deleting Views...")
    
    for v in all_views:
        print(f"Deleting Dataview: {v}...")
        this_view = get_kx_dataview(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=v)

        if this_view is None:
            print(f"Not Found: {v}")
        elif this_view['status'] not in delete_status:
            print(f"Cannot Delete: {v} Status: {this_view['status']}")
        else:
            print(f"delete: {v}")
            client.delete_kx_dataview(environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=v)
            print("** Deleted View **")

    for v in all_views:
        wait_for_dataview_status(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=v, status='DELETED', show_wait=True)
            
    print(f"** Deleted {len(all_views)} Views **")
            
# Volume
#-------------------------------------------------------------------------------------------
if DELETE_VOLUME:
    print()
    print(f"Deleting Volume: {VOLUME_NAME}...")
    this_volume = get_kx_volume(client=client, environmentId=ENV_ID, volumeName=VOLUME_NAME)

    if this_volume is None:
        print(f"Not Found: {VOLUME_NAME}")
    elif this_volume['status'] not in delete_status:
        print(f"Cannot Delete: {VOLUME_NAME} Status: {this_volume['status']}")
    else:
        print(f"Delete: {VOLUME_NAME}")
        client.delete_kx_volume(environmentId=ENV_ID, volumeName=VOLUME_NAME)
        wait_for_volume_status(client=client, environmentId=ENV_ID, volumeName=VOLUME_NAME, status='DELETED', show_wait=True)
        print("** Deleted Volume **")

# Scaling Group
#-------------------------------------------------------------------------------------------
if DELETE_SCALING_GROUP:
    print()
    print(f"Deleting Scaling Group: {SCALING_GROUP_NAME}...")
    this_sg = get_kx_scaling_group(client=client, environmentId=ENV_ID, scalingGroupName=SCALING_GROUP_NAME)

    if this_sg is None:
        print(f"Not Found: {SCALING_GROUP_NAME}")
    elif this_sg['status'] not in delete_status:
        print(f"Cannot Delete: {SCALING_GROUP_NAME} Status: {this_sg['status']}")
    else:
        print(f"Delete: {SCALING_GROUP_NAME}")
        client.delete_kx_scaling_group(environmentId=ENV_ID, scalingGroupName=SCALING_GROUP_NAME)
        wait_for_scaling_group_status(client=client, environmentId=ENV_ID, scalingGroupName = SCALING_GROUP_NAME, status='DELETED', show_wait=True)
        print("** Deleted Scaling Group **")

# Database
#-------------------------------------------------------------------------------------------
if DELETE_DATABASE:
    print()
    print(f"Deleting Database: {DB_NAME}...")
    this_db = get_kx_database(client=client, environmentId=ENV_ID, databaseName=DB_NAME)

    if this_db is None:
        print(f"Not Found: {DB_NAME}")
    else:
        print(f"delete: {DB_NAME}")
        client.delete_kx_database(environmentId=ENV_ID, databaseName=DB_NAME)    
        print("** Deleted Database **")

print("\nDone Deleting")

Deleting Clusters...
	Delete: dbmaint_cluster_maint
	Deleted: dbmaint_cluster_maint
	Delete: dbmaint_cluster_query
	Deleted: dbmaint_cluster_query
** Deleted 2 Clusters **

Deleting Views...
Deleting Dataview: dbmaintdb_DBVIEW_MAINT...
delete: dbmaintdb_DBVIEW_MAINT
** Deleted View **
Deleting Dataview: dbmaintdb_DBVIEW_QUERY...
delete: dbmaintdb_DBVIEW_QUERY
** Deleted View **
Dataview: dbmaintdb_DBVIEW_MAINT status is DELETING, total wait 0:00:00, waiting 30 sec ...
Dataview: dbmaintdb_DBVIEW_MAINT status is DELETING, total wait 0:00:30, waiting 30 sec ...
Dataview: dbmaintdb_DBVIEW_MAINT status is DELETING, total wait 0:01:00, waiting 30 sec ...
Dataview: dbmaintdb_DBVIEW_MAINT status is DELETING, total wait 0:01:30, waiting 30 sec ...
Dataview: dbmaintdb_DBVIEW_MAINT not found
Dataview: dbmaintdb_DBVIEW_QUERY status is DELETING, total wait 0:00:00, waiting 30 sec ...
Dataview: dbmaintdb_DBVIEW_QUERY status is DELETING, total wait 0:00:30, waiting 30 sec ...
Dataview: dbmaintdb_DBVI

In [4]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2024-11-12 18:08:04.413738
